In [ ]:
import gymnasium
from gymnasium import spaces
import numpy as np
import pandas as pd
import heapq
import pygame
import matplotlib.pyplot as plt
from collections import defaultdict
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
import os

class RadarTaskSchedulerEnv(gymnasium.Env):
    metadata = {"render_modes": ["human"], "render_fps": 1}

    def __init__(self, task_file="dataset2.csv", render_mode=None):
        super(RadarTaskSchedulerEnv, self).__init__()

        self.tasks_df = pd.read_csv(task_file)
        self.num_tasks = len(self.tasks_df)

        self.priority_queue = []
        self.entry_count = 0

        self.current_task_data = None

        self.scheduled_tasks_log = []

        # Observation space: [Duration, Deadline, Init_Power]
        self.observation_space = spaces.Box(low=0, high=100, shape=(3,), dtype=np.float32)

        # Action space: Single Discrete action representing combinations of (delay, compress, radar_choice)
        self.action_space = spaces.Discrete(3 * 3 * 3)

        self.render_mode = render_mode
        self.screen = None
        self.font = None
        self.clock = None
        self.width = 800
        self.height = 600

        self._last_info = {}
        self._last_reward = 0

    def reset(self,*, seed=None, options=None):
        super().reset(seed=seed)

        self.priority_queue = []
        self.entry_count = 0

        # Populate priority queue: (negative_priority, entry_count, task_data_dict)
        for index, row in self.tasks_df.iterrows():
            negative_priority = -row['Priority']
            heapq.heappush(self.priority_queue, (negative_priority, self.entry_count, row.to_dict()))
            self.entry_count += 1
        
        # Reset the log for a new episode
        self.scheduled_tasks_log = [] 
        self._last_info = {}
        self._last_reward = 0

        obs = self._get_observation()
        
        if self.render_mode == "human":
            self._init_render()
            self.render()

        return obs, {}

    def step(self, action_index):
        if not self.priority_queue:
            # If no tasks left, return dummy observation and signal done
            return np.array([0, 0, 0], dtype=np.float32), 0, True, False, {
                "task_dropped": False, 
                "task_id": "N/A", 
                "message": "No tasks left to process.",
                "dropped_tasks_count": 0 # Explicitly add for terminal state
            }

        _priority, _entry_count, task_to_process = heapq.heappop(self.priority_queue)
        
        delay = action_index // 9
        compress = (action_index % 9) // 3
        radar_choice = action_index % 3
        
        tns = task_to_process['Request_Time'] + delay
        Pn = task_to_process['Init_Power'] + compress

        task_dropped = (tns > task_to_process['Deadline']) or \
                       (Pn > task_to_process['Max_Power'])

        reward = 0.0

        if not task_dropped:
            reward += 1.0
        else:
            reward -= 1.0

        if not task_dropped:
            time_to_deadline = task_to_process['Deadline'] - tns
            if time_to_deadline < 2:
                reward -= 0.1
            elif time_to_deadline < 0: # Should ideally not happen if not task_dropped
                reward -= 0.2
        elif tns > task_to_process['Deadline'] + 5:
             reward -= 0.2

        if not task_dropped:
            power_margin = task_to_process['Max_Power'] - Pn
            if power_margin < 2:
                reward -= 0.1
            elif power_margin < 0: # Should ideally not happen if not task_dropped
                reward -= 0.2
        elif Pn > task_to_process['Max_Power'] + 5:
            reward -= 0.2

        if delay == 0 and not task_dropped:
            reward += 0.05

        if compress == 0 and not task_dropped:
            reward += 0.05

        info = { 
            "task_id": task_to_process["Task_ID"],
            "task_dropped": task_dropped,
            "delay_action": delay,
            "compress_action": compress,
            "radar_choice_action": radar_choice,
            "calculated_tns": tns,
            "calculated_Pn": Pn,
            "original_deadline": task_to_process['Deadline'],
            "original_max_power": task_to_process['Max_Power']
        }
        self._last_info = info
        self._last_reward = reward

        self.scheduled_tasks_log.append({
            'Task_ID': task_to_process['Task_ID'],
            'Priority': task_to_process['Priority'],
            'Action_Delay': delay,
            'Action_Compress': compress,
            'Action_Radar': radar_choice,
            'Dropped': task_dropped,
            'Final_TNS': tns,
            'Final_Pn': Pn,
            'Original_Deadline': task_to_process['Deadline'],
            'Original_Max_Power': task_to_process['Max_Power']
        })

        done = not self.priority_queue

        # If the episode is ending, add the total dropped tasks count to the info
        if done:
            info['dropped_tasks_count'] = sum(1 for task in self.scheduled_tasks_log if task['Dropped'])
        else:
            # If not done, this will be 0 or not present for interim steps,
            # but we only care about the final step's info for episode summaries.
            # We can still add it here, it just won't be the final count.
            info['dropped_tasks_count'] = 0


        next_obs = self._get_observation()
            
        if self.render_mode == "human":
            self.render()

        return next_obs, reward, done, False, info

    def _get_observation(self):
        if self.priority_queue:
            _priority, _entry_count, task_dict = self.priority_queue[0] 
            self.current_task_data = task_dict
            return np.array([task_dict['Duration'], task_dict['Deadline'], task_dict['Init_Power']], dtype=np.float32)
        else:
            self.current_task_data = None
            return np.array([0, 0, 0], dtype=np.float32)

    def _init_render(self):
        if self.screen is None:
            pygame.init()
            pygame.display.set_caption("Radar Task Scheduler")
            self.screen = pygame.display.set_mode((self.width, self.height))
            self.font = pygame.font.Font(None, 30)
            self.clock = pygame.time.Clock()

    # --- NEW HELPER METHOD FOR RENDERING ---
    def _draw_text(self, text, color, y_offset, x_offset=10):
        text_surface = self.font.render(text, True, color)
        self.screen.blit(text_surface, (x_offset, y_offset))

    def render(self):
        if self.render_mode != "human":
            return

        self._init_render()

        self.screen.fill((0, 0, 0)) # Black background

        y_offset = 20
        self._draw_text(f"Tasks Remaining: {len(self.priority_queue)}", (255, 255, 255), y_offset)
        y_offset += 40

        if self._last_info:
            self._draw_text("--- Last Task Processed ---", (255, 255, 0), y_offset)
            y_offset += 30
            task_id = self._last_info.get('task_id')
            priority_val = 'N/A'
            if task_id is not None and not self.tasks_df[self.tasks_df['Task_ID'] == task_id].empty:
                priority_val = self.tasks_df[self.tasks_df['Task_ID'] == task_id]['Priority'].iloc[0]

            self._draw_text(f"Task ID: {task_id} (Prio: {priority_val})", (200, 200, 200), y_offset)
            y_offset += 25
            self._draw_text(f"Action: Delay {self._last_info.get('delay_action')}, Compress {self._last_info.get('compress_action')}, Radar {self._last_info.get('radar_choice_action')}", (150, 250, 150), y_offset)
            y_offset += 25
            self._draw_text(f"TNS: {self._last_info.get('calculated_tns')} (Deadline: {self._last_info.get('original_deadline')})", (200, 200, 200), y_offset)
            y_offset += 25
            self._draw_text(f"Pn: {self._last_info.get('calculated_Pn')} (Max: {self._last_info.get('original_max_power')})", (200, 200, 200), y_offset)
            y_offset += 25
            
            task_dropped_color = (255, 0, 0) if self._last_info.get('task_dropped', False) else (0, 255, 0)
            self._draw_text(f"Dropped: {self._last_info.get('task_dropped')}", task_dropped_color, y_offset)
            y_offset += 25
            
            reward_color = (0, 255, 0) if self._last_reward > 0 else (255, 0, 0) if self._last_reward < 0 else (255, 255, 255)
            self._draw_text(f"Reward: {self._last_reward:.2f}", reward_color, y_offset)
            y_offset += 40

        if self.current_task_data:
            self._draw_text("--- Next Task to Process (Highest Priority) ---", (0, 255, 255), y_offset)
            y_offset += 30
            self._draw_text(f"Task ID: {self.current_task_data['Task_ID']}", (200, 200, 200), y_offset)
            y_offset += 25
            self._draw_text(f"Priority: {self.current_task_data['Priority']}", (200, 200, 200), y_offset)
            y_offset += 25
            self._draw_text(f"Duration: {self.current_task_data['Duration']}", (200, 200, 200), y_offset)
            y_offset += 25
            self._draw_text(f"Request Time: {self.current_task_data['Request_Time']}", (200, 200, 200), y_offset)
            y_offset += 25
            self._draw_text(f"Deadline: {self.current_task_data['Deadline']}", (200, 200, 200), y_offset)
            y_offset += 25
            self._draw_text(f"Init Power: {self.current_task_data['Init_Power']}", (200, 200, 200), y_offset)
            y_offset += 25
            self._draw_text(f"Max Power: {self.current_task_data['Max_Power']}", (200, 200, 200), y_offset)
            y_offset += 25
        else:
            self._draw_text("--- All tasks processed ---", (0, 255, 0), y_offset)

        pygame.display.flip()
        self.clock.tick(self.metadata["render_fps"])

    def close(self):
        if self.screen is not None:
            pygame.quit()
            self.screen = None
            self.font = None
            self.clock = None

# --- Custom Callback for Episode Logging ---
class EpisodeLoggerCallback(BaseCallback):
    # Added total_episodes_target to __init__
    def __init__(self, output_file, tasks_per_episode, total_episodes_target, verbose=0):
        super(EpisodeLoggerCallback, self).__init__(verbose)
        self.output_file = output_file
        self.tasks_per_episode = tasks_per_episode
        self.total_episodes_target = total_episodes_target # Store the target for logging
        self.episode_num = 0
        self.reward_log = []
        self.task_drop_log = []

    def _on_step(self) -> bool:
        """
        Called by the RL model at each step.
        """
        # Check if an episode just finished (dones is True for the first environment)
        if self.locals['dones'][0]:
            self.episode_num += 1
            
            total_reward_episode = 0
            task_drops_episode = 0

            # Access the info dictionary for the *last step* of the completed episode
            # This is where Monitor puts the episode summary information.
            last_step_info = self.locals['infos'][0]

            if 'episode' in last_step_info:
                episode_stats = last_step_info['episode']
                total_reward_episode = episode_stats['r']
                
                # Retrieve the 'dropped_tasks_count' directly from the info dictionary
                # This key is now explicitly added by the RadarTaskSchedulerEnv when an episode ends
                task_drops_episode = last_step_info.get('dropped_tasks_count', 0)
                
                self.reward_log.append(total_reward_episode)
                self.task_drop_log.append(task_drops_episode)

                # Access exploration rate (epsilon) for logging
                epsilon = self.model.exploration_rate if hasattr(self.model, 'exploration_rate') else 'N/A'

                # CORRECTED: Use self.total_episodes_target for the denominator
                log_message = f"Episode {self.episode_num}/{self.total_episodes_target} — Total Reward: {total_reward_episode:.2f} — Tasks Dropped: {task_drops_episode} — Epsilon (approx): {epsilon:.3f}"
                print(log_message)
                self.output_file.write(log_message + "\n")
        return True

    def _on_rollout_end(self) -> None:
        pass # No additional logging needed here, handled by _on_step

    def _on_training_end(self) -> None:
        pass


if __name__ == "__main__":
    # Create a dummy dataset2.csv if it doesn't exist for demonstration
    # if not os.path.exists("dataset2.csv"):
    #     print("dataset2.csv not found. Creating a dummy one for demonstration.")
    #     # Creating a larger dataset for more meaningful training
    #     num_dummy_tasks = 100 
    #     dummy_data = {
    #         'Task_ID': [f'Task_{i}' for i in range(1, num_dummy_tasks + 1)],
    #         'Priority': np.random.randint(1, 11, num_dummy_tasks),
    #         'Duration': np.random.randint(5, 20, num_dummy_tasks),
    #         'Request_Time': np.random.randint(0, 10, num_dummy_tasks),
    #         'Deadline': np.random.randint(15, 40, num_dummy_tasks),
    #         'Init_Power': np.random.randint(10, 30, num_dummy_tasks),
    #         'Max_Power': np.random.randint(35, 60, num_dummy_tasks)
    #     }
    #     pd.DataFrame(dummy_data).to_csv("dataset2.csv", index=False)


    enable_rendering = input("Enable Pygame rendering during training? (Y/N): ").strip().lower()
    if enable_rendering == 'y':
        render_mode_choice = "human"
        print("Pygame rendering ENABLED. Close the Pygame window to stop training prematurely.")
    else:
        render_mode_choice = None
        print("Pygame rendering DISABLED. Training will run silently.")

    env = RadarTaskSchedulerEnv(task_file="dataset2.csv", render_mode=render_mode_choice)
    env = Monitor(env)

    dqn_hyperparams = {
        "learning_rate": 0.0005,      # Increased learning rate
        "buffer_size": 10000,
        "learning_starts": 500,       # Reduced learning starts
        "batch_size": 32,
        "gamma": 0.995,               # Increased discount factor
        "train_freq": (1, "step"),    # Train every step
        "gradient_steps": 1,
        "target_update_interval": 100,
        "exploration_fraction": 0.1,
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "max_grad_norm": 10,
        "seed": 42
    }

    model = DQN("MlpPolicy", env, verbose=0, **dqn_hyperparams, device="auto")

    episodes_to_train = 1000 # This is your target number of episodes
    tasks_per_episode = len(pd.read_csv("dataset2.csv"))
    total_timesteps_for_training = episodes_to_train * tasks_per_episode

    output_filename = "dqn_learning_results.txt"
    
    with open(output_filename, "w") as f_out:
        f_out.write("--- DQN Training Log ---\n\n")
        print("Starting DQN training...")
        f_out.write("Starting DQN training...\n\n")

        # Pass total_episodes_target to the callback
        episode_logger_callback = EpisodeLoggerCallback(
            output_file=f_out, 
            tasks_per_episode=tasks_per_episode,
            total_episodes_target=episodes_to_train # Pass the target here
        )

        # Train the model with the callback
        model.learn(total_timesteps=total_timesteps_for_training, callback=episode_logger_callback)

        print("\nTraining complete!")
        f_out.write("\nTraining complete!\n")

        # Save model only at the very end
        model_path = "dqn_radar_scheduler_final_model"
        model.save(model_path)
        print(f"Final model saved to {model_path}")
        f_out.write(f"Final model saved to {model_path}\n")

        # Retrieve logs from the callback
        reward_log = episode_logger_callback.reward_log
        task_drop_log = episode_logger_callback.task_drop_log

        window_size = 50

        # Plotting Rewards
        if not reward_log:
            print("\nReward log is empty. No reward plot will be generated.")
            f_out.write("\nReward log is empty. No reward plot will be generated.\n")
        else:
            moving_avg = []
            x_moving_avg = []
            if len(reward_log) >= window_size:
                moving_avg = np.convolve(reward_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg = range(window_size - 1, len(reward_log))

            plt.figure(figsize=(14,7))
            plt.plot(reward_log, label='Episode Rewards', color='orange', alpha=0.7, linewidth=1)
            
            if moving_avg.size > 0:
                plt.plot(x_moving_avg, moving_avg, label=f'Moving Average ({window_size} episodes)', color='blue', linewidth=2)
            
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Total Reward', fontsize=12)
            plt.title('Total Reward over Episodes (DQN)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_reward_plot.png")
            plt.close()
            print(f"Reward plot saved as dqn_reward_plot.png")
            f_out.write(f"Reward plot saved as dqn_reward_plot.png\n")


        # Plotting Task Drops
        if not task_drop_log:
            print("\nTask drop log is empty. No task drop plot will be generated.")
            f_out.write("\nTask drop log is empty. No task drop plot will be generated.\n")
        else:
            moving_avg_drops = []
            x_moving_avg_drops = []
            if len(task_drop_log) >= window_size:
                moving_avg_drops = np.convolve(task_drop_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg_drops = range(window_size - 1, len(task_drop_log))

            plt.figure(figsize=(14,7))
            plt.plot(task_drop_log, label='Tasks Dropped per Episode', color='red', alpha=0.8, linewidth=1)
            
            if moving_avg_drops.size > 0:
                plt.plot(x_moving_avg_drops, moving_avg_drops, label=f'Moving Average ({window_size} episodes)', color='darkred', linewidth=2)
            
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Tasks Dropped', fontsize=12)
            plt.title('Number of Tasks Dropped per Episode (DQN)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_task_drop_plot.png")
            plt.close()
            print(f"Task drop plot saved as dqn_task_drop_plot.png")
            f_out.write(f"Task drop plot saved as dqn_task_drop_plot.png\n")


        # --- Final Evaluation Episode for Schedule and Metrics ---
        f_out.write("\n\n--- Running Final Evaluation Episode for Schedule and Metrics ---\n")
        print("\n--- Running Final Evaluation Episode for Schedule and Metrics ---")

        obs, _ = env.reset()
        done = False
        
        unwrapped_env_eval = env.env 
        unwrapped_env_eval.scheduled_tasks_log = [] # Clear any previous log for this eval run

        while not done:
            action_index, _states = model.predict(obs, deterministic=True) # Use deterministic policy for evaluation
            obs, reward, done, _, info = env.step(action_index)

        # The scheduled_tasks_log from the unwrapped_env_eval now contains the full run for this episode
        f_out.write("\n--- Final Task Schedule (from last evaluation) ---\n")
        print("\n--- Final Task Schedule (from last evaluation) ---")
        if unwrapped_env_eval.scheduled_tasks_log and len(unwrapped_env_eval.scheduled_tasks_log) == unwrapped_env_eval.num_tasks:
            schedule_df = pd.DataFrame(unwrapped_env_eval.scheduled_tasks_log)
            f_out.write(schedule_df.to_string(index=False))
            print(schedule_df.to_string(index=False))
        else:
            f_out.write("No full schedule generated in the final evaluation. This might mean the environment did not complete a full episode (e.g., due to an empty task queue).\n")
            print("No full schedule generated in the final evaluation. This might mean the environment did not complete a full episode (e.g., due to an empty task queue).")

        # Performance Metrics Table from the final evaluation
        f_out.write("\n\n--- Performance Metrics (from last evaluation) ---\n")
        print("\n--- Performance Metrics (from last evaluation) ---")

        if unwrapped_env_eval.scheduled_tasks_log:
            total_tasks_attempted = len(unwrapped_env_eval.scheduled_tasks_log)
            completed_tasks = sum(1 for task in unwrapped_env_eval.scheduled_tasks_log if not task['Dropped'])
            dropped_tasks_count = sum(1 for task in unwrapped_env_eval.scheduled_tasks_log if task['Dropped'])

            if total_tasks_attempted > 0:
                percent_tasks_completed = (completed_tasks / total_tasks_attempted) * 100
                percent_tasks_dropped = (dropped_tasks_count / total_tasks_attempted) * 100
            else:
                percent_tasks_completed = 0.0
                percent_tasks_dropped = 0.0

            metrics = {
                "Total Tasks in Dataset": unwrapped_env_eval.num_tasks,
                "Tasks Attempted (in last evaluation)": total_tasks_attempted,
                "Tasks Successfully Completed": completed_tasks,
                "Tasks Dropped": dropped_tasks_count,
                "Percentage of Tasks Completed": f"{percent_tasks_completed:.2f}%",
                "Percentage of Tasks Dropped": f"{percent_tasks_dropped:.2f}%"
            }

            metrics_df = pd.DataFrame(metrics.items(), columns=['Metric', 'Value'])
            f_out.write(metrics_df.to_string(index=False))
            print(metrics_df.to_string(index=False))
        else:
            f_out.write("No metrics could be calculated from the final evaluation as no tasks were processed.\n")
            print("No metrics could be calculated from the final evaluation as no tasks were processed.")

    env.close()
    print("Environment closed.")